In [1]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt # for making figures
%matplotlib inline

words = open('names.txt', 'r').read().splitlines()
print(words[:8])
print(f'Number of words in the file : {len(words)}')

# Building VOCAB
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s, i in stoi.items()}
vocab_size = len(itos)
#print(itos)
print(f"vocab size is : {vocab_size}")

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']
Number of words in the file : 32084
vocab size is : 27


In [2]:
# build the dataset
block_size = 3 # context length: how many characters do we take to predict the next one?

def build_dataset(words):  
  X, Y = [], []
  
  for w in words:
    context = [0] * block_size
    for ch in w + '.':
      ix = stoi[ch]
      X.append(context)
      Y.append(ix)
      context = context[1:] + [ix] # crop and append

  X = torch.tensor(X)
  Y = torch.tensor(Y)
  print(X.shape, Y.shape)
  return X, Y

import random
random.seed(42)
random.shuffle(words)
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))

Xtr,  Ytr  = build_dataset(words[:n1])     # 80%
Xdev, Ydev = build_dataset(words[n1:n2])   # 10%
Xte,  Yte  = build_dataset(words[n2:])  

torch.Size([182603, 3]) torch.Size([182603])
torch.Size([23085, 3]) torch.Size([23085])
torch.Size([22834, 3]) torch.Size([22834])


### Linear Layer

In [3]:
class Linear:
  
  def __init__(self, fan_in, fan_out, bias=True):
    self.weight = torch.randn((fan_in, fan_out), generator=g) / fan_in**0.5
    self.bias = torch.zeros(fan_out) if bias else None
  
  def __call__(self, x):
    self.out = x @ self.weight
    if self.bias is not None:
      self.out += self.bias
    return self.out
  
  def parameters(self):
    return [self.weight] + ([] if self.bias is None else [self.bias])

### BatchNorm1d Layer

In [6]:
class BatchNorm1d:
  
  def __init__(self, dim, eps=1e-5, momentum=0.1):
    self.eps = eps
    self.momentum = momentum
    self.training = True
    # parameters (trained with backprop)
    self.gamma = torch.ones(dim)
    self.beta = torch.zeros(dim)
    # buffers (trained with a running 'momentum update')
    self.running_mean = torch.zeros(dim)
    self.running_var = torch.ones(dim)
  
  def __call__(self, x):
    # calculate the forward pass
    if self.training:
      xmean = x.mean(0, keepdim=True) # batch mean
      xvar = x.var(0, keepdim=True) # batch variance
    else:
      xmean = self.running_mean
      xvar = self.running_var
    xhat = (x - xmean) / torch.sqrt(xvar + self.eps) # normalize to unit variance
    self.out = self.gamma * xhat + self.beta
    # update the buffers
    if self.training:
      with torch.no_grad():
        self.running_mean = (1 - self.momentum) * self.running_mean + self.momentum * xmean
        self.running_var = (1 - self.momentum) * self.running_var + self.momentum * xvar
    return self.out
  
  def parameters(self):
    return [self.gamma, self.beta]

### Activation

In [7]:
class Tanh:
  def __call__(self, x):
    self.out = torch.tanh(x)
    return self.out
  def parameters(self):
    return []

## Model

In [10]:
n_embd = 10 # the dimensionality of the character embedding vectors
n_hidden = 100 # the number of neurons in the hidden layer of the MLP
g = torch.Generator().manual_seed(2147483647) # for reproducibility

C = torch.randn((vocab_size, n_embd), generator=g)
layers = [
#----Layer 1------
  Linear(n_embd * block_size, n_hidden, bias=False), 
  #BatchNorm1d(n_hidden), 
  Tanh(),
# ---Layer 2 -----
  Linear(n_hidden, n_hidden, bias=False), 
  #BatchNorm1d(n_hidden),
  Tanh(),
# ---Layer 3 ----- 
  Linear(n_hidden, n_hidden, bias=False), 
  #BatchNorm1d(n_hidden),
  Tanh(),
# ---Layer 4 ----- 
  Linear(n_hidden, n_hidden, bias=False),
  #BatchNorm1d(n_hidden),
  Tanh(),
# ---Layer 5 -----
  Linear(n_hidden, n_hidden, bias=False),
  #BatchNorm1d(n_hidden), 
  Tanh(),
# ---Layer 6 -----
  Linear(n_hidden, vocab_size, bias=False), 
  #BatchNorm1d(vocab_size),
]

with torch.no_grad():
  # last layer: make less confident
  
  layers[-1].weight *= 0.1
  # all other layers: apply gain
  for layer in layers[:-1]:
    if isinstance(layer, Linear):
      layer.weight *= 5/3

parameters = [C] + [p for layer in layers for p in layer.parameters()]
print(sum(p.nelement() for p in parameters)) # number of parameters in total
for p in parameters:
  p.requires_grad = True

45970


In [11]:
# same optimization as last time
max_steps = 200000
batch_size = 32
lossi = []

for i in range(max_steps):

  # minibatch construct
  ix = torch.randint(0, Xtr.shape[0], (batch_size,), generator=g)
  Xb, Yb = Xtr[ix], Ytr[ix] # batch X,Y
  
  # forward pass
  emb = C[Xb] # embed the characters into vectors
  x = emb.view(emb.shape[0], -1) # concatenate the vectors
  for layer in layers: 
        x = layer(x)
  loss = F.cross_entropy(x, Yb) # loss function
  
  # backward pass
  for layer in layers: 
        layer.out.retain_grad()
  for p in parameters:
    p.grad = None
  loss.backward()

  # update
  lr = 0.1 if i < 100000 else 0.01 # step learning rate decay
  for p in parameters:
    p.data += -lr * p.grad

  # track stats
  if i % 10000 == 0: # print every once in a while
    print(f'loss -->{i:7d}/{max_steps:7d}: {loss.item():.4f}')
  lossi.append(loss.log10().item())
  #break

loss -->      0/ 200000: 3.3009
loss -->  10000/ 200000: 2.0381
loss -->  20000/ 200000: 2.4680
loss -->  30000/ 200000: 2.1650
loss -->  40000/ 200000: 2.2115
loss -->  50000/ 200000: 1.7809
loss -->  60000/ 200000: 2.3042
loss -->  70000/ 200000: 1.9355
loss -->  80000/ 200000: 2.3423
loss -->  90000/ 200000: 1.9930
loss --> 100000/ 200000: 1.8873
loss --> 110000/ 200000: 2.1533
loss --> 120000/ 200000: 2.1022
loss --> 130000/ 200000: 2.1315
loss --> 140000/ 200000: 1.9278
loss --> 150000/ 200000: 1.5714
loss --> 160000/ 200000: 1.7359
loss --> 170000/ 200000: 1.9304
loss --> 180000/ 200000: 1.9094
loss --> 190000/ 200000: 1.8691
